<a href="https://colab.research.google.com/github/corinnakeum1/DS-4002/blob/main/CorinnaLinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q0

What makes a model "linear"? "Linear" in what?


How do you interpret the coefficient for a dummy/one-hot-encoded variable? (This is a trick question, and the trick involves how you handle the intercept of the model.)


Can linear regression be used for classification? Explain why, or why not.
What are signs that your linear model is over-fitting?


Clearly explain multi-colinearity using the two-stage least squares technique.
What are two ways to incorporate nonlinear relationships between your target/response/dependent/outcome variable
 and your features/control/response/independent variables
?


What is the interpretation of the intercept? A slope coefficient for a variable? The coefficient for a dummy/one-hot-encoded variable?